# # Classifiers comparison on texts with naive Bayes assumption

In this session of laboratory we compare two models for categorical data probabilistic modeling: 
1. multivariate Bernoulli 
2. multinomial on a dataset 

We adopt a dataset on Twitter messages labelled with emotions (Joy vs Sadness).

The following program shows the loading of the data from a file.

Data are loaded into a matrix X adopting a sparse matrix representation, in order to save space and time.
Sparse matrix representation (in the csr format) represents in three "parallel" arrays the value of the matrix cells that are different from zero and the indices of those matrix cells.
The arrays are called: 
- data
- row
- col

- data[i] stores the value of the matrix cell #i whose indexes are contained in row[i] and col[i] 
- row[i] stores the index of the row in the matrix of the cell #i, 
- col[i] stores the index of the column of the cell #i.


The data file is in csv format.
Any Twitter message has been preprocessed by a Natural Language pipeline which eliminated stop words and substituted the interesting document elements with an integer identifier.  
The interesting document elements might be words, emoji or emoticons. The elements could be repeated in the same document and are uniquely identified in the documents by the same integer number (named "element_id" in the program). This "element_id" number will be used as the index of the column of the data matrix, for the purposes of storage of data.

Each row of the CSV file reports the content of a document (a Twitter message).It is formed as a list of integer number pairs, followed by a string which is the label of the document ("Joy" or "Sadness").
The first number of the pair is the identifier of a document element (the "element_id"); 
the second number of the pair is the count (frequency) of that element in that document.

The dataset has:

tot_n_docs (or rows in the file) =n_rows=11981

n_features (total number of distinct words in the corpus)=11288

The following program reads the data file and loads in a sparse way the matrix using the scipy.sparse library


____


Quindi se ho capito bene dovrebbe essere:
1. Struttura generale del file:
Due classi di sentimenti:
I messaggi sono classificati in due categorie:
Joy : Rappresenta un sentimento positivo.
Sadness : Rappresenta un sentimento negativo.
Il file è organizzato come segue:
Prime 5988 righe: Messaggi classificati come Joy.
Successive 5994 righe: Messaggi classificati come Sadness.
Numero totale di messaggi: 11.981 messaggi

2. Struttura di ogni riga:
Ogni riga del file rappresenta un singolo messaggio, ed è composta dai seguenti elementi:
Sequenza di coppie <wordID, count>:
Ogni coppia rappresenta un identificatore di parola e il numero di volte in cui quella parola appare nel messaggio.
Esempio: 38,3 significa che la parola con ID 38 appare 3 volte nel messaggio.
Class Label:
Dopo le coppie <wordID, count>, troviamo l'etichetta della classe, che può essere:
Joy oppure Sadness .
Questa etichetta indica il sentimento espresso dal messaggio.

Il dataset è letto riga per riga. Ogni riga rappresenta un messaggio e contiene coppie <wordID, count> che vengono trasformate in una matrice sparsa. L'etichetta della classe (Joy o Sadness) viene salvata in un array di target separato. La matrice sparsa è poi utilizzata per rappresentare i dati in modo efficiente, riducendo lo spazio occupato grazie al fatto che la maggior parte dei valori sono zero.

La matrice sparsa:

Non salva gli 0, ma solo i valori non nulli e i loro indici.
È ideale per dati testuali (o simili), dove ogni documento utilizza solo una piccola parte del vocabolario totale.
Permette di risparmiare memoria e accelerare i calcoli, rendendola essenziale in problemi di machine learning con dataset grandi e sparsi come questo.





In [1]:

from numpy import ndarray, zeros
import numpy as np
import scipy
from scipy.sparse import csr_matrix

from collections import Counter



class_labels = ["Joy","Sadness"]

# Numero di feature (parole distinte nel vocabolario)
n_features=11288 # number of columns in the matrix = number of features (distinct elements in the documents)

# Numero di documenti (messaggi di Twitter)
n_rows=11981 # number rows of the matrix

# Numero di elementi diversi da zero nella matrice
n_elements=71474 # number of the existing values in the matrix (not empty, to be loaded in the matrix in a sparse way)

#path_training="/Users/meo/Documents/Didattica/Laboratorio-15-16-Jupyter/"
path_training="./"
file_name="joy_sadness6000.txt"

# declare the row and col arrays with the indexes of the matrix cells (non empty) to be loaded from file
# they are needed because the matrix is sparse and we load in the matrix only the elements which are present
row=np.empty(n_elements, dtype=int) #Contiene gli indici delle righe in cui si trovano i valori diversi da zero.  Ogni indice corrisponde al documento
col=np.empty(n_elements, dtype=int) #Contiene gli indici delle colonne (cioè le feature, o parole) per i valori diversi da zero
data=np.empty(n_elements, dtype=int)#Contiene i valori diversi da zero da inserire nella matrice (

#print("n_row="+str(row))
#print("n_col="+str(col))
#print("n_data="+str(data))


row_n=0 # number of current row to be read and managed
cur_el=0 # position in the three arrays: row, col and data
twitter_labels=[] # list of class labels (target array) of the documents (twitter) that will be read from the input file
twitter_target=[] # list of 0/1 for class labels
with open(path_training + file_name, "r") as fi:
    for line in fi:
        el_list=line.split(',')  # list of integers read from a row of the file
        l=len(el_list)
        last_el=el_list[l-1] # I grab the last element in the list which is the class label
        class_name=last_el.strip() # eliminate the '\n'
        twitter_labels.append(class_name)
        # twitter_labels contains the labels (Joy/Sadness); twitter_target contains 0/1 for the respective labels
        if (class_name==class_labels[0]):
           twitter_target.append(0)
        else:
           twitter_target.append(1)
        i=0 # I start reading all the doc elements from the beginning of the list
        while(i<(l-1)):
            element_id=int(el_list[i]) # identifier of the element in the document equivalent to the column index
            element_id=element_id-1 # the index starts from 0 (the read id starts from 1)
            i=i+1
            value_cell=int(el_list[i]) # make access to the following value in the file which is the count of the element in the documento 
            i=i+1
            row[cur_el]=row_n # load the data in the three arrays: the first two are the row and col indexes; the last one is the matrix cell value
            col[cur_el]=element_id
            data[cur_el]=value_cell
            cur_el=cur_el+1
        row_n=row_n+1
fi.close
print("final n_row="+str(row))# loads the matrix by means of the indexes and the values in the three arrays just filled

#Creazione della matrice sparsa
twitter_data=csr_matrix((data, (row, col)), shape=(n_rows, n_features)).toarray()

print(csr_matrix((data, (row, col)), shape=(n_rows, n_features)))

print("resulting matrix:")
print(twitter_data)
print(twitter_labels)
print(twitter_target)


# Conta la distribuzione delle classi
distribution = Counter(twitter_target)
print(distribution)
distribution = Counter(twitter_labels)
print(distribution)

# Calcola le percentuali
total = sum(distribution.values())
percentages = {k: (v / total) * 100 for k, v in distribution.items()}
print(percentages)


final n_row=[0 0 0 ... 0 0 0]
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 71414 stored elements and shape (11981, 11288)>
  Coords	Values
  (0, 0)	1
  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (1, 4)	1
  (1, 5)	1
  (1, 6)	1
  (1, 7)	1
  (1, 8)	1
  (1, 9)	1
  (1, 10)	1
  (1, 11)	1
  (2, 12)	1
  (2, 13)	1
  (2, 14)	1
  (2, 15)	1
  (2, 16)	1
  (2, 17)	1
  (2, 18)	1
  (2, 19)	1
  (2, 20)	1
  (2, 21)	1
  (3, 19)	1
  (3, 22)	1
  (3, 23)	1
  :	:
  (11977, 460)	1
  (11977, 677)	1
  (11977, 1604)	1
  (11977, 1816)	1
  (11977, 2037)	2
  (11977, 7764)	1
  (11977, 8234)	1
  (11978, 65)	1
  (11978, 66)	1
  (11978, 890)	1
  (11978, 2637)	1
  (11978, 7764)	1
  (11978, 11030)	1
  (11978, 11287)	1
  (11979, 211)	1
  (11979, 1024)	1
  (11979, 1173)	1
  (11979, 1352)	1
  (11979, 2989)	1
  (11979, 7764)	1
  (11980, 1)	1
  (11980, 104)	2
  (11980, 1173)	1
  (11980, 1321)	1
  (11980, 7761)	1
resulting matrix:
[[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ...

Scrivi un programma nella cella seguente che divida la matrice dei dati in training e test set (tramite selezione casuale) e preveda la classe (Gioia/Tristezza) dei messaggi in base alle parole.
Considera i due possibili modelli: Bernoulli multivariato e Bernoulli multinomiale.
Trova l'accuratezza dei modelli e verifica se le differenze osservate sono significative.

# Fitting

Adatto i modelli usando cross validation


In [49]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.model_selection import cross_validate, train_test_split

# Suddivisione del dataset in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(
    twitter_data, # Matrice dei dati (feature dei tweet)
    twitter_target, # Etichette (es.: sentiment )
    test_size=0.25, # Il 25% dei dati sarà usato come set di test
    random_state=42, 
    shuffle=True
)

# Creazione del modello Naive Bayes con distribuzione di probabilità Bernoulli
multivariate_bernoulli_model = BernoulliNB(force_alpha=True)


# Creazione del modello Naive Bayes Multinomiale
multinomial_bernoulli_model = MultinomialNB(force_alpha=True)

# Addestramento dei modelli
multivariate_bernoulli_model.fit(X_train, y_train)
multinomial_bernoulli_model.fit(X_train, y_train)



# Valutazione delle prestazioni dei modelli
multivariate_scores = cross_validate(
    multivariate_bernoulli_model, twitter_data, twitter_target, scoring="f1", cv=20
)
multinomial_scores = cross_validate(
    multinomial_bernoulli_model, twitter_data, twitter_target, scoring="f1", cv=20
)




#multivariate_scores = cross_validate(
#    multivariate_bernoulli_model, twitter_data, twitter_target, scoring="accuracy", cv=20
#)
#multinomial_scores = cross_validate(
#    multinomial_bernoulli_model, twitter_data, twitter_target, scoring="accuracy", cv=20
#)




print(multivariate_scores)
print(multinomial_scores)

{'fit_time': array([4.28300619, 5.31322122, 4.36535811, 4.06117225, 3.62825394,
       3.58390832, 3.59911036, 3.64580631, 3.61800504, 3.99200583,
       3.52559805, 4.08814597, 4.10041523, 4.01407337, 4.03097439,
       4.16026735, 3.81808949, 3.59701824, 3.5779078 , 4.26414156]), 'score_time': array([0.07099962, 0.11600447, 0.09091353, 0.07497525, 0.05499911,
       0.05999756, 0.06111431, 0.06192255, 0.06799507, 0.09801197,
       0.05991745, 0.08501077, 0.07299709, 0.06391191, 0.06900263,
       0.06999898, 0.05800033, 0.06206441, 0.06400418, 0.0779922 ]), 'test_score': array([0.95723684, 0.96078431, 0.96880131, 0.96039604, 0.93548387,
       0.96369637, 0.9472    , 0.96039604, 0.95961228, 0.94041868,
       0.95315024, 0.95765472, 0.95284553, 0.95253682, 0.95253682,
       0.95666132, 0.96880131, 0.96103896, 0.93851133, 0.9398374 ])}
{'fit_time': array([3.70546317, 3.01499915, 3.19473767, 3.1167624 , 2.98799491,
       3.06733131, 2.97801876, 3.07115507, 3.00535393, 2.87941265,
  

# Predictions

matrice di confusione dei 2 modelli e  punteggi F1_scores


In [50]:
from sklearn.metrics import confusion_matrix, f1_score,accuracy_score

mv_y_pred = multivariate_bernoulli_model.predict(X_test) # Predizioni sul set di test
print(f"Multivariate Predictions:\n{mv_y_pred}\n")
mv_confusion_matrix = confusion_matrix(y_test, mv_y_pred)# Calcolo della matrice di confusione
print(f"Multivariate Confusion Matrix:\n{mv_confusion_matrix}\n")
print(f"The Multivariate F_1 Score:\t{f1_score(y_test, mv_y_pred)}\n")
#print(f"The Multivariate Accuracy Score:\t{accuracy_score(y_test, mv_y_pred)}\n")

mn_y_pred = multinomial_bernoulli_model.predict(X_test)# Predizioni sul set di test
print(f"Multinomial Predictions:\n{mn_y_pred}\n")
mn_confusion_matrix = confusion_matrix(y_test, mn_y_pred)# Calcolo della matrice di confusione
print(f"Multinomial Confusion Matrix:\n{mn_confusion_matrix}")
print(f"The Multinomial F_1 Score:\t{f1_score(y_test, mn_y_pred)}")
#print(f"The Multinomial Accuracy Score:\t{accuracy_score(y_test, mn_y_pred)}")

Multivariate Predictions:
[1 0 1 ... 0 1 1]

Multivariate Confusion Matrix:
[[1405  120]
 [  28 1443]]

The Multivariate F_1 Score:	0.9512195121951219

Multinomial Predictions:
[1 0 1 ... 0 1 1]

Multinomial Confusion Matrix:
[[1396  129]
 [  28 1443]]
The Multinomial F_1 Score:	0.9484061781137035


# Test di ipotesi

Ora controllerò se i due modelli sono statisticamente uguali:
 - Utilizzando la cross-validation  estraggo una metrica Utilizzando un test T di Student verifico se la differenza della metrica è 0 (i modelli hanno prestazioni identiche)

Confronto statisticamente i due modelli (MultinomialNB e BernoulliNB).
verifico se uno dei due modelli ha prestazioni significativamente migliori dell'altro

In [51]:
from scipy.stats import ttest_1samp


# Calcolo delle differenze tra i punteggi dei due modelli per ogni fold
score_differences = [
    mn_s - mv_s # Calcola la differenza: punteggio Multinomial - punteggio Bernoulli
    for mn_s, mv_s in zip(
        multinomial_scores["test_score"], 
        multivariate_scores["test_score"]
    )
]
print(score_differences)

# Calcola la media delle differenze
mu = np.average(score_differences)
print(f"\nmean of observations: {mu}\n")


# Test t a un campione: verifica se la media delle differenze è significativamente diversa da 0
# popmean=0 perché l'ipotesi nulla (H0) assume che i modelli abbiano le stesse prestazioni
#Se nessuno dei due test ha un P-value ≤ 0.05, significa che non ci sono prove sufficienti per concludere che uno dei due modelli sia significativamente migliore.


# Test "greater"
g = ttest_1samp(score_differences, popmean=0, alternative="greater") # verifica se il modello Multinomial ha performance significativamente migliori del modello Bernoulli.

# Test "less"
l = ttest_1samp(score_differences, popmean=0, alternative="less") # verifica se il modello Bernoulli ha performance significativamente migliori del modello Multinomial.

# Stampa dei risultati dei due test t unilaterali
print(g)
print(l)

[np.float64(-0.0015718174747212377), np.float64(-0.004686752749880507), np.float64(-0.0013746035311800187), np.float64(-0.00013113894170868612), np.float64(-0.004727027167419795), np.float64(0.005002147661224088), np.float64(-0.003021371610845369), np.float64(-0.0014469427238289478), np.float64(-0.001683475278531521), np.float64(-0.004934808581372407), np.float64(-0.00646203554119551), np.float64(-0.0015571621514260947), np.float64(0.00015285079917248812), np.float64(-0.0029433289422911013), np.float64(-0.0013967597306706603), np.float64(-0.004738239288801127), np.float64(-0.001481052190991372), np.float64(-0.0031101584499643176), np.float64(0.0001983505585134715), np.float64(0.0005812811751305658)]

mean of observations: -0.001966602208039403

TtestResult(statistic=np.float64(-3.4387727332703126), pvalue=np.float64(0.998623981576072), df=np.int64(19))
TtestResult(statistic=np.float64(-3.4387727332703126), pvalue=np.float64(0.00137601842392801), df=np.int64(19))


### Risultati

Ho confrontato le prestazioni di **Multinomial Naive Bayes** e **Bernoulli Naive Bayes** utilizzando l'**F1-score** e una **cross validation a 20 fold**. La **media delle differenze** tra i punteggi dei due modelli è risultata essere  $\mu = -0,002%$, indicando che il modello **Bernoulli** ha ottenuto prestazioni leggermente migliori.

Per verificare se questa differenza è significativa, ho utilizzato un **test t a un campione**, con l'ipotesi nulla \( H_0 \) che la **media delle differenze** sia pari a 0. I risultati del test sono:



* $P(\Mu \ge \mu | H_0) = 99.86 \%$
* $P(\Mu \le \mu | H_0) = 0.14 \%$   < 0.05

Poiché la probabilità di ottenere una differenza così estrema sotto \( H_0 \) è solo **0,14%**, rifiutiamo l'ipotesi nulla e concludiamo che il **modello Bernoulli** funziona meglio del **modello Multinomial** con una certezza del **99,86%**.
